In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33923
2,Huelva,Confirmados PDIA 14 días,831
3,Huelva,Tasa PDIA 14 días,"161,93464154178926"
4,Huelva,Confirmados PDIA 7 días,399
5,Huelva,Total Confirmados,34108
6,Huelva,Curados,32242
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33923.0
/tmp/ipykernel_24082/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9276.0
/tmp/ipykernel_24082/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_24082/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_24082/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_24082/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1424 personas en los últimos 7 días 

Un positivo PCR cada 660 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33923.0,399.0,831.0,513170.0,77.752012,161.934642,45.0
Huelva-Costa,19579.0,255.0,520.0,289548.0,88.068300,179.590258,24.0
Condado-Campiña,11015.0,121.0,258.0,156231.0,77.449418,165.140081,19.0
Huelva (capital),9276.0,101.0,218.0,143837.0,70.218372,151.560447,15.0
Isla Cristina,2252.0,48.0,70.0,21393.0,224.372458,327.209835,6.0
Niebla,262.0,13.0,13.0,4116.0,315.840622,315.840622,6.0
Almonte,1944.0,24.0,38.0,24507.0,97.931203,155.057739,4.0
Aljaraque,1192.0,11.0,30.0,21474.0,51.224737,139.703828,2.0
Sierra de Huelva-Andévalo Central,2970.0,18.0,47.0,67391.0,26.709798,69.742250,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,29.0,13.0,13.0,1043.0,1246.404602,1246.404602,0.0
Alosno,201.0,12.0,21.0,3933.0,305.110603,533.943555,0.0
Isla Cristina,2252.0,48.0,70.0,21393.0,224.372458,327.209835,6.0
Zarza-Perrunal (La),30.0,3.0,4.0,1253.0,239.425379,319.233839,1.0
Niebla,262.0,13.0,13.0,4116.0,315.840622,315.840622,6.0
Bonares,313.0,7.0,19.0,6060.0,115.511551,313.531353,0.0
Cerro de Andévalo (El),160.0,3.0,6.0,2327.0,128.921358,257.842716,0.0
Ayamonte,1388.0,21.0,54.0,21104.0,99.507202,255.875663,1.0
Lucena del Puerto,275.0,7.0,8.0,3261.0,214.658080,245.323520,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,579.0,1.0,7.0,7939.0,12.596045,88.172314,0.0,0.142857
San Juan del Puerto,597.0,4.0,21.0,9411.0,42.503453,223.143130,0.0,0.190476
Manzanilla,110.0,1.0,5.0,2118.0,47.214353,236.071766,0.0,0.200000
Palos de la Frontera,740.0,7.0,26.0,11742.0,59.615057,221.427355,1.0,0.269231
Palma del Condado (La),1203.0,4.0,14.0,10801.0,37.033608,129.617628,1.0,0.285714
Cartaya,1425.0,13.0,40.0,20083.0,64.731365,199.173430,0.0,0.325000
Rosal de la Frontera,34.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Moguer,1435.0,17.0,49.0,21867.0,77.742717,224.081950,1.0,0.346939
Aljaraque,1192.0,11.0,30.0,21474.0,51.224737,139.703828,2.0,0.366667
